# 06 - Custom Sequences - Part 1

#### Lecture

We're going to be creating our own sequence types. For part 1, we'll focus on immutable sequence types only. At it's most basic, an immutable sequence type should support two things:

- returning the **length** of the sequence,
- given an **index**, return the element at that index.

If an object provides this functionality, then in theory we should be able to:

- retrieve elements **by index** using square brackets `[]`,
- **iterate** through the elements using Python's native looping mechanisms such as `for` loops, list comprehensions, etc.

**How does Python do it?** 

The sequence types must at the minimum implement:

- `__len__`: We'll find out that this is not necessary but still recommended
- `__getitem__`: This is the main one for taking a single integer argument - the index. However, it may also choose to handle a **slice** type argument. We decide.


#### The `__getitem__` method

**This method should return an element of the sequence based on the specified index or raise an `IndexError` exception if the index is out of bounds.**

If we implement these **two** things then Python can actually utilise our `__getitem__` method to iterate through our sequence, e.g. by using the `for` loop.

(We may choose to support negative indices and slices but this is optional.)

Python's `list` already supports `__getitem__` as an equivalent approach to the `[]`. Negative indices are valid so long as they are not out of bounds.

In [18]:
my_list = ['a', 'b', 'c', 'd', 'e', 'f']

print(my_list.__getitem__(0))
print(my_list.__getitem__(slice(0, 6, 2)))
print(my_list.__getitem__(-1))

a
['a', 'c', 'e']
f


This include the `IndexError`:

In [16]:
print(my_list.__getitem__(100))

IndexError: list index out of range

**How would we implement a `for` loop ourselves?**

In [19]:
my_list = [1, 2, 3, 4, 5]

for item in my_list:
    print(item**2)

1
4
9
16
25


In [21]:
my_list = [1, 2, 3, 4, 5]
index = 0

while True:
    try:
        item = my_list.__getitem__(index)
    except IndexError:
        break
    
    print(item**2)
    index += 1

1
4
9
16
25


The approach above of using `while True` paired with `except` is more Pythonic than something like `while index < len(our_sequence)`.

When we implement `__getitem__` in our class, the `for` loop (or list comprehension) notation will function, where the `x` in `for x in sequence` will be passed to `__getitem__`. Again, it  will keep iterating until it hits an `IndexError`

In [31]:
class Silly:
    def __init__(self, n):
        self.n = n
        
    def __getitem__(self, value): 
        print(f'{value=}')
        
        if value < 0 or value >= self.n:
            raise IndexError
        
        else:
            return 'This is a silly element'

In [32]:
silly = Silly(3)

for e in silly:
    print(e)

value=0
This is a silly element
value=1
This is a silly element
value=2
This is a silly element
value=3


The small difference we notice with the list comprehension approach below is that the `__getitem__` is being called multiple times as the list is being built up, and then finally, the list object is returned. But of course, we won't normally have a `print` statement within the `__getitem__` method.

In [34]:
[e for e in silly]

value=0
value=1
value=2
value=3


['This is a silly element',
 'This is a silly element',
 'This is a silly element']

**This object `silly` *is* a sequence**

We can also use the slice notation such as `silly[0:5:2]` and Python will use the `__getitem__` method where the value is a `slice` object with the specified parameters. 

It won't work with our current setup because the line `if value < 0 or value >= self.n:` contains comparison operators which doesn't support slice objects of course. 

The appropriate approach is to check the instance/type of `value` and deal with it differently depending on whether it's a **slice** object or an integer. We'll demonstrate this below.

Now let's work through a more practical example.

**Fibonacci Sequence Example**

Here, we're going to implement our own Fibonacci sequence type and we want to support slicing.

We'll make our sequence type bounded (i.e. we'll have to specify the size of the sequence). But we are not going to pre-generate the entire sequence of Fibonacci numbers, we'll only generate the ones that are being requested as needed.

So, when we create our object, we'll have to specify the maximum Fibonacci number that will be possible for this sequence (but we won't generate that value or any other Fibonacci value during the creation of the object). See below.

```
f = Fib(10)    # Creating our Fibonacci sequence that can generate up to 10 values.
f[3:5]         # It can handle slices. In this case, return a list of the element 3 and 4 of the Fibonacci sequence (i.e. 4th and 5th).
f[::-1]        # Return a list of all Fibonacci values in reverse order. In this case, the 10th value and below. 
```

Here are the steps:


1. First we'll need to handle requesting a single index e.g. `f[3]` for the 4th Fibonacci value. To do this, we just need `__getitem__` which will allow the `[]` notation. 


2. To actually calculate the nth Fibonacci number, we'll use memoisation as well (see lecture on decorators and memoisation if you need to refresh your memory on that).

3. To be able to use `list(f)`, we need to raise an `IndexError` exception when the index is out of bounds as this is one of the two essential criteria for iterating through a sequence. 

4. We also need to remap negative indices (for example `-1` should correspond to the last element of the sequence, and so on). This is done with:     

```
if s < 0:
    s = self._n + s
```

5. Now let's handle slice requests. If you recall from Section 05 - Slicing, you might think it to be difficult to implement all those rules. But remember, the slice method `indices(<length of sequence>)` returns a tuple of 3 values which correspond to the equivalent start, stop and step values. These can be converted into a range object which we can iterate through, e.g. using a list comprehension or a 'for' loop, computing the corresponding Fibonacci value at each step. We can go further by generating the full list to show us which indices are going to be asked for.

   For example:
   
   ```
   my_slice = slice(2, -7, -2)          # When applied to a sequence s, it'd be equivalent to s[2:-7:-2]. Hard to see which indices we are requiring.

   list(range(*my_slice.indices(6)))    # If we pass in the length of our sequence (6) then it outputs that we're asking for...
   
   > [2, 0]                             # ...index 2 and index 0. So find _fib(2) and then _fib(0), but we've not implemented this extra detail below.
   ```
   
   We're done.

In [27]:
from functools import lru_cache

class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            if s < 0:
                s = self._n + s
            if s < 0 or s > self._n - 1:
                raise IndexError
            return self._fib(s)                                 # this is basically our 'else' statement.
            
        else:
            # slice being requested
            idx = s.indices(self._n)                            # the argument required for s.indices is the length of the sequence which is self._n
            rng = range(*idx)                                   # This will create a range sequence object e.g. range(0, 10, 2), corresponding to the indices
            return [self._fib(n) for n in rng]                  # in the Fibonacci sequence that we want.
            
    @staticmethod
    @lru_cache(2**32)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)

In [28]:
f = Fib(10)

print(f[0])
print(f[9])
print(list(f))
print(list(item for item in f))
print(f[2:9:2])

1
55
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[2, 5, 13, 34]


**Sidenote:**

A static method is bound to a class rather than the objects for that class. This means that a static method can be called without an object for that class. This also means that static methods cannot modify the state of an object as they are not bound to it.

Static methods have a very clear use-case. When we need some functionality not w.r.t an Object but w.r.t the complete class, we make a method static. This is pretty much advantageous when we need to create Utility methods as they aren’t tied to an object lifecycle usually. Finally, note that in a static method, we don’t need the self to be passed as the first argument.

**Sidenote:**

You may be wondering why the last line is `Fib._fib(n-1)` as opposed to `self._fib(n-1)` that we see in the `__getitem__` method. We could do it that way but things will start to look convoluted very quickly. For starters `def _fib(n)` will need to become `def _fib(self, n)`. Then, in `__getitem__`, `return self._fib(s)` will need to become `return self._fib(self, s)`. The code below is the version with these changes made.


```    
def __getitem__(self, s):
    if isinstance(s, int):
        # single item requested
        if s < 0:
            s = self._n + s
        if s < 0 or s > self._n - 1:
            raise IndexError
                
        return self._fib(self, s)                                 # this is basically our 'else' statement.
    else:
        # slice being requested
        print(f'requesting [{s.start}:{s.stop}:{s.step}]')
        idx = s.indices(self._n)                            # the argument required for s.indices is the length of the sequence which is self._n
        rng = range(*idx)
        print(f'\trange({idx[0]}, {idx[1]}, {idx[2]}) --> {list(rng)}')  # \t is tab special character
            
@staticmethod
@lru_cache(2**32)
def _fib(self, n):
    if n < 2:
        return 1
    else:
        return self._fib(self, n-1) + self._fib(self, n-2)
```

One thing I want to point out here: we did not need to use inheritance! There was no need to inherit from another sequence type. All we really needed was to implement the `__getitem__` and `__len__` methods.

The other thing I want to mention, is that I would not use recursion for production purposes for a Fibonacci sequence, even with memoization - partly because of the cost of recursion and the limit to the recursion depth that is possible.

Also, when we look at generators, and more particularly generator expressions, we'll see better ways of doing this as well.

I really wanted to show you a simple example of how to create your own sequence types.